<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/alpaca_LightGBM_execution_adapter_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install alpaca-trade-api

In [4]:
!pip install alpaca-trade-api python-dotenv ta

In [18]:
import os
from dotenv import load_dotenv
import pickle
import lightgbm as lgb
import pandas as pd
import ta
from datetime import datetime, timedelta
from alpaca_trade_api.rest import REST, TimeFrame, APIError
import time
import warnings
warnings.filterwarnings("ignore")  # Optional: suppress version warnings

In [5]:
from google.colab import files
uploaded = files.upload()  # Prompts you to upload the .env file for the key

Saving Alpaca_keys.env.txt to Alpaca_keys.env.txt


In [7]:
import os

# Rename to .env so dotenv can recognize it
os.rename("Alpaca_keys.env.txt", ".env")

In [39]:
from google.colab import files
uploaded = files.upload()  # Upload: model_ABT.txt, features_ABT.txt, scaler_ABT.pkl

import lightgbm as lgb
import pickle

# Load LightGBM model
model = lgb.Booster(model_file='model_ABT.txt')

# Load feature list
with open('features_ABT.txt', 'r') as f:
    model_features = [line.strip() for line in f]

# Load scaler
with open('scaler_ABT.pkl', 'rb') as f:
    scaler = pickle.load(f)

print("Model, features, and scaler loaded for ABT.")


Saving features_ABT.txt to features_ABT (1).txt
Saving model_ABT.txt to model_ABT (1).txt
Saving scaler_ABT.pkl to scaler_ABT (1).pkl
Model, features, and scaler loaded for ABT.


In [40]:
# Load .env credentials
load_dotenv(override=True)
API_KEY = os.getenv("APCA_API_KEY_ID")
SECRET_KEY = os.getenv("APCA_API_SECRET_KEY")
BASE_URL = "https://paper-api.alpaca.markets"  # or live URL

# Create API object
api = REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

# Check account
account = api.get_account()
print("Account status:", account.status)


Account status: ACTIVE


In [41]:
#Load Model, Scaler, Features Dynamically
def load_model_artifacts(ticker):
    model = lgb.Booster(model_file=f'model_{ticker}.txt')
    with open(f'features_{ticker}.txt', 'r') as f:
        features = [line.strip() for line in f]
    with open(f'scaler_{ticker}.pkl', 'rb') as f:
        scaler = pickle.load(f)
    return model, features, scaler

# Example for ABT
ticker = "ABT"
model, model_features, scaler = load_model_artifacts(ticker)
print(f"Model artifacts loaded for {ticker}")

Model artifacts loaded for ABT


In [42]:
def add_features(df):
    df['SMA_50'] = ta.trend.sma_indicator(df['close'], window=50)
    df['EMA_20'] = ta.trend.ema_indicator(df['close'], window=20)

    volatility = df['close'].rolling(14).std()
    rsi = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    macd = ta.trend.macd_diff(df['close'])
    signal_line = ta.trend.macd_signal(df['close'])

    df['RSI_voladj'] = rsi * volatility
    df['MACD_voladj'] = macd * volatility
    df['Signal_Line'] = signal_line
    df['ATR_voladj'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range() * volatility
    df['OBV'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    df['CCI_voladj'] = ta.trend.cci(df['high'], df['low'], df['close'], window=20) * volatility

    return df.dropna()

In [43]:
from datetime import datetime, timedelta

end = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
start = (datetime.utcnow() - timedelta(days=45)).replace(microsecond=0).isoformat() + "Z"
bars = api.get_bars(ticker, TimeFrame.Hour, start=start, end=end, feed="iex").df
bars = bars.reset_index()

#Check before feature engineering
if len(bars) < 100:
    raise ValueError(f"Not enough raw data to compute indicators for {ticker}. Got only {len(bars)} rows.")

bars = add_features(bars)

#Check after feature engineering
if len(bars) < 10:
    raise ValueError(f"All rows dropped after feature engineering for {ticker}.")

print(f"Raw bars before features: {len(bars)}")
bars = add_features(bars)
print(f"Bars remaining after feature engineering: {len(bars)}")

Raw bars before features: 154
Bars remaining after feature engineering: 105


In [44]:
# Correctly parse features from comma-separated line
with open('features_ABT.txt', 'r') as f:
    model_features = f.read().strip().split(',')

# Check availability after feature engineering
bars = add_features(bars)
print("After add_features():")
print("Available columns:", bars.columns.tolist())

missing = [f for f in model_features if f not in bars.columns]
if missing:
    raise ValueError(f"Still missing features after add_features(): {missing}")


After add_features():
Available columns: ['timestamp', 'close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap', 'SMA_50', 'EMA_20', 'RSI_voladj', 'MACD_voladj', 'Signal_Line', 'ATR_voladj', 'OBV', 'CCI_voladj']


In [45]:
#Make Prediction
if not all(f in bars.columns for f in model_features):
    raise ValueError(f"Missing features in live data for {ticker}.")

X_live = bars[model_features].copy().dropna()
X_scaled = scaler.transform(X_live)

preds = model.predict(X_scaled)
signal = int(preds[-1])
print(f"Prediction for {ticker}: {signal} (1 = Buy, 0 = Sell)")

Prediction for ABT: 0 (1 = Buy, 0 = Sell)


In [46]:
#Make Prediction
if not all(f in bars.columns for f in model_features):
    raise ValueError(f"Missing features in live data for {ticker}.")

X_live = bars[model_features].copy().dropna()
X_scaled = scaler.transform(X_live)

preds = model.predict(X_scaled)
signal = int(preds[-1])
print(f"Prediction for {ticker}: {signal} (1 = Buy, 0 = Sell)")

#Execute Trade Logic
try:
    clock = api.get_clock()
    if not clock.is_open:
        print("Market is closed.")
    else:
        try:
            position = api.get_position(ticker)
            has_position = float(position.qty) > 0
        except APIError:
            has_position = False
            print("ℹNo current position in", ticker)

        if signal == 1 and not has_position:
            api.submit_order(symbol=ticker, qty=1, side='buy', type='market', time_in_force='gtc')
            print(f"BUY order submitted for {ticker}")
        elif signal == 0 and has_position:
            api.submit_order(symbol=ticker, qty=1, side='sell', type='market', time_in_force='gtc')
            print(f"SELL order submitted for {ticker}")
        else:
            print("No action taken for", ticker)

except Exception as e:
    print("Error during trading:", e)


# Log Portfolio + Equity

positions = api.list_positions()
for p in positions:
    print(f"{p.symbol}: {p.qty} shares (${p.market_value})")

history = api.get_portfolio_history(period="1D", timeframe="5Min")
print("Recent Equity:", history.equity[-5:])

print(f"Strategy complete for {ticker}.")

Prediction for ABT: 0 (1 = Buy, 0 = Sell)
ℹNo current position in ABT
No action taken for ABT
AAPL: 2 shares ($419.66)
Recent Equity: [100010.8, 100009.96, 100009.94, 100010.85, 100010.9]
Strategy complete for ABT.
